In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#single node of the tree
class Node:
    def __init__(self, feature = None, threshold = None, left_node = None, right_node = None, label = None):
        #define optional parameters
        self.feature = feature
        self.threshold = threshold
        self.left_node = left_node
        self.right_node = right_node
        self.label = label
        
    #check if a node is a leaf     
    def is_leaf(self):
        return self.label is not None

In [14]:
class D_tree:
    def __init__(self, max_depth = 10, min_samples_split= 2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None
    
    #check if we reached the defined limits
    def is_done(self, y, depth):
        
        if (depth >= self.max_depth
            or len(y) <= self.min_samples_split
            or len(np.unique(y)) == 1):
            return True
        
        return False
    
    #calculate the Enthropy in a node
    def entropy(self, y):
        #proportions of the classes
        p = np.bincount(y)/len(y)
        ent = -np.sum([r*np.log(r) for r in p if r > 0])
        return ent
    
    def split(self, X, thresh):
        #find the indexes of the left and right childs
        left_idx = np.argwhere(X <= thresh).flatten()
        right_idx = np.argwhere(X > thresh).flatten()
        
        return left_idx, right_idx
    
    def info_gain(self, X, y, thresh):
        
        left, right = self.split(X, thresh)
        p = len(y)
        l = len(left)
        r = len(right)
        parent = self.entropy(y)
        left_child = (l/p)*self.entropy(y[left])
        right_child = (r/p)*self.entropy(y[right])
        
        return parent - left_child - right_child
        
    def best_split(self, X, y):
        
        best_combination = {"gain": -1, 
                           "best_feat" : None, 
                           "best_thresh": None}
        #loop through features
        for f in range(X.shape[1]):
            x_feat = X[:,f]
            threshs = np.unique(x_feat)
            #loop through possible threshold
            for t in threshs:
                gain = self.info_gain(x_feat, y, t)
                #check the improvment of the information gain
                if gain > best_combination["gain"]:
                    best_combination["gain"] = gain
                    best_combination["best_feat"] = f
                    best_combination["best_thresh"] = t
                    
        return best_combination["best_feat"], best_combination["best_thresh"]  
    
    #building the tree
    def build_tree(self, X, y, depth = 0):
        
        #if we rach the limits return the leaf node
        if self.is_done(y, depth):
            label = np.argmax(np.bincount(y))
            return Node(label = label)
        
        #find the best split
        best_feat, best_thresh = self.best_split(X, y)
        
        left, right = self.split(X[:,best_feat], best_thresh)
        left_child = self.build_tree(X[left,:], y[left], depth + 1)
        right_child = self.build_tree(X[right,:], y[right], depth + 1)
        
        return Node(best_feat, best_thresh, left_child, right_child)
    
    def fit(self, X, y):
        self.root = self.build_tree(X, y, depth = 0)
    
    #find the path for an element
    def traverse_tree(self, x, node):
        
        if node.is_leaf():
            return node.label
        
        val = x[node.feature]
        
        if val <= node.threshold:
            return self.traverse_tree(x, node.left_node)
        else:
            return self.traverse_tree(x, node.right_node)
        
            
    
    def predict(self, X):
        return np.array([self.traverse_tree(e, self.root) for e in X])
    
    

In [15]:
from sklearn.datasets import load_iris

iris = load_iris()

X = iris['data']
y = iris['target']

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
model = D_tree()
model.fit(X_train, y_train)
preds = model.predict(X_test)

In [18]:
def accuracy(y_pred, y_true):
    acc = np.sum(y_pred == y_true)/len(y_true)
    return acc

print("accuracy of the model : ", accuracy(preds, y_test))

accuracy of the model :  1.0
